<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/Bank-Churner-Prediction/blob/master/Bank_Churner_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기 및 EDA


In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

In [ ]:
ls

라이브러리 모음

In [ ]:

import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install -U pandas-profiling

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split, RandomizedSearchCV,cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, roc_auc_score, classification_report,f1_score as f1
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn import metrics

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/BankChurners.csv')
data.head()

In [ ]:
pr = data.profile_report()
pr

### 변수 설명

- CLIENTNUM : 고객 번호. 계정을 보유한 고객의 고유 식별자
- **Attrition_Flag(Target) : 고객 이탈 여부, 1 - Churner/ 0 - non Churner**
- Customer_Age : 고객의 나이
- Gender : 성별, M=Male, F=Female
- Dependent_count : 부양가족수
- Education_Level : 고객의 학력
- Marital_Status : Married, Single, Divorced, Unknown
- Income_Category : 연간 소득 (< 40K, 40K - 60K, 60K - 80K, 80K-120K, >)
- Card_Category : 카드 등급 (Blue, Silver, Gold, Platinum)
- Months_on_book : 은행과 거래 기간
- Total_Relationship_Count : 고객이 보유한 총 상품 수
- Months_Inactive_12_mon : 최근 12개월 동안 비활성 개월 수
- Contacts_Count_12_mon : 최근 12개월 동안 연락한 횟수
- Credit_Limit : 신용 한도
- Total_Revolving_Bal : 총 리볼빙(일부결제금액이월약정) 잔액
- Avg_Open_To_Buy : Open to Buy Credit Line (Average of last 12 months)
- Total_Amt_Chng_Q4_Q1 : 거래 금액의 변화 (1분기 대비 4분기)
- Total_Trans_Amt : 최근 12개월 총 거래 금액
- Total_Trans_Ct : 최근 12개월 총 거래 횟 수
- Total_Ct_Chng_Q4_Q1 : Change in Transaction Count (Q4 over Q1)
- Avg_Utilization_Ratio : 평균 카드 사용률

### EDA

In [ ]:
# Customer_Age 분포

# 그래프 영역 나누기
f = make_subplots(rows=2, cols=1) 

# boxplot과 histogram 그래프 x,y축 지정, boxmean=평균값표시
t1 = go.Box(x=data['Customer_Age'], name='Box Plot for Customer_age', boxmean=True)
t2 = go.Histogram(x=data['Customer_Age'], name='Histogram Customer_age')

# plotly에서는 add_trace() 함수로 지정
f.add_trace(t1, row=1, col=1) 
f.add_trace(t2, row=2, col=1)

f.update_layout(height=600, width=1000, title_text = 'Customer_Age')

Customer_Age는 정규분포 <br>
아직 경제활동을 하고있지 않는 70세 이상의 초고령층의 데이터가 필요할까?

In [ ]:
data.sort_values(by='Customer_Age', ascending=True)

In [ ]:
# Card_Category 분포
%matplotlib inline 

plt.figure(figsize=(8,8))
sns.countplot(data=data, x='Card_Category', hue='Gender')
plt.title('Card Rating by Gender')
plt.show()

In [ ]:
data['Card_Category'].value_counts()

In [ ]:
# Dependent_count 분포

# 그래프 영역 나누기
f = make_subplots(rows=2, cols=1) 

# boxplot과 histogram 그래프 x,y축 지정, boxmean=평균값표시
t1 = go.Box(x=data['Dependent_count'], name='Box Plot for Dependent_count', boxmean=True)
t2 = go.Histogram(x=data['Dependent_count'], name='Histogram Dependent_count')

# plotly에서는 add_trace() 함수로 지정
f.add_trace(t1, row=1, col=1) 
f.add_trace(t2, row=2, col=1)

f.update_layout(height=600, width=1000, title_text = 'Dependent_count')

In [ ]:
# Education_Level 분포
ex.pie(data,names='Education_Level',title='Propotion Of Education Levels',hole=0.2)

In [ ]:
data['Education_Level'].value_counts()

In [ ]:
# Marital_Stattus 분포

px.pie(data,names='Marital_Status',title='Propotion Of Different Marriage Statuses',hole=0.33)

In [ ]:
# Income_Category 분포

px.pie(data,names='Income_Category',title='Income_Category',hole=0.33)

In [ ]:
# 'Atrittion_Flag' 타겟 변수 분포 
px.pie(data,names='Attrition_Flag',title='churn vs not churn customers',hole=0.33)

# Data Preprocessing

In [ ]:
data.head()

In [ ]:
# 결측 값이 unknown 으로 되어 있어서 na값 확인되지 않음
data.isna().sum()

In [ ]:
# 필요없는 변수 제거
data.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
                   'CLIENTNUM'], inplace=True)

In [ ]:
data

In [ ]:
# Unknown 비율 20% 이하 제거

data = data[data['Marital_Status'] != 'Unknown'] # 7.4%
data = data[data['Income_Category'] != 'Unknown'] # 11%
data = data[data['Education_Level'] != 'Unknown'] # 15%

print(data['Marital_Status'].value_counts())
print('\n')
print(data['Income_Category'].value_counts())
print('\n')
print(data['Education_Level'].value_counts())

### Encoding

In [ ]:
# encoding
data['Attrition_Flag'] = data['Attrition_Flag'].replace({'Attrited Customer':1, 'Existing Customer':0})
data['Gender'] = data['Gender'].replace({'F':1, 'M':0})

In [ ]:
# Oridinal Encoding

Income_Category_map = {
    'Less than $40K' : 0,
    '$40K - $60K'    : 1,
    '$60K - $80K'    : 2,
    '$80K - $120K'   : 3,
    '$120K +'        : 4
}


Card_Category_map = {
    'Blue'     : 0,
    'Silver'   : 1,
    'Gold'     : 2,
    'Platinum' : 3
}

Education_Level_map = {
    'Uneducated'    : 0,
    'High School'   : 1,
    'College'       : 2,
    'Graduate'      : 3,
    'Post-Graduate' : 4,
    'Doctorate'     : 5
    }

data.loc[:, 'Income_Category'] = data['Income_Category'].map(Income_Category_map)
data.loc[:, 'Card_Category'] = data['Card_Category'].map(Card_Category_map)
data.loc[:, 'Education_Level'] = data['Education_Level'].map(Education_Level_map)

data.head()

In [ ]:
# marital_statis onehot encoding
data = pd.get_dummies(data)
data.head()

In [ ]:
# Customer_Age 70세 이상 데이터 제외(2명)
data = data[data['Customer_Age'] < 70]
data.sort_values(by='Customer_Age', ascending=False)

In [ ]:
data.info()

# Modeling

In [ ]:
# 타겟 설정 및 데이터 나누기
target = 'Attrition_Flag'
features = data.drop(columns= target).columns

train, test = train_test_split(data, test_size = 0.2, random_state = 2021, stratify = data[target])
train, val = train_test_split(train, test_size = 0.2, random_state = 2021, stratify = train[target])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
# SMOTE로 데이터 불균형 해결
sm = SMOTE(ratio='auto', kind='regular', random_state=2021)

# train데이터를 넣어 복제함
X_train_over,y_train_over = sm.fit_sample(X_train,y_train)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

### Model 1. Random Forest

In [ ]:
pipe = Pipeline([
                 ('rf', RandomForestClassifier(
                                      random_state = 2021,
                                      n_jobs = -1))])

param = {  
            'rf__n_estimators': randint(100,500), 
            'rf__max_depth' : randint(3,10),
            'rf__min_samples_split' : randint(3,20),
            'rf__min_samples_leaf': randint(3,20),
            }


clf = RandomizedSearchCV(
    pipe, 
    param_distributions=param, 
    n_iter=50, 
    cv=3, 
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train_over, y_train_over)

In [ ]:
model = clf.best_estimator_
y_pred = model.predict(X_test)
RF_opt_fpr, RF_opt_tpr, RF_opt_thresholds = metrics.roc_curve(y_test, y_pred)
RF_opt_AUC = metrics.auc(RF_opt_fpr, RF_opt_tpr)

fig = px.area(
    x=RF_opt_fpr, y=RF_opt_tpr,
    title='AUC of Random Forest :'' %0.4f'% RF_opt_AUC,
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=600
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

### Model.2 Adaboost

In [ ]:
pipe2 = Pipeline([
                 ('ada', AdaBoostClassifier(
                                      random_state = 2021))])

param2 = {  
            'ada__n_estimators': randint(100,1000), 
            'ada__learning_rate' : uniform(0.01,1)
            }


clf2 = RandomizedSearchCV(
    pipe2, 
    param_distributions=param2, 
    n_iter=50, 
    cv=3, 
    verbose=1,
    n_jobs=-1
)

clf2.fit(X_train_over, y_train_over)

In [ ]:
model2 = clf2.best_estimator_
y_pred2 = model2.predict(X_test)
RF_opt_fpr2, RF_opt_tpr2, RF_opt_thresholds2 = metrics.roc_curve(y_test, y_pred2)
RF_opt_AUC2 = metrics.auc(RF_opt_fpr2, RF_opt_tpr2)

fig = px.area(
    x=RF_opt_fpr2, y=RF_opt_tpr2,
    title='AUC of Adaboost:'' %0.4f'% RF_opt_AUC2,
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=600
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

## Model 3. LightGBM

In [ ]:
pipe3 = Pipeline([
                 ('lgbm', LGBMClassifier(
                                      random_state = 2021,
                                      objective = 'binary',
                                      eval_metric = 'auc',
                                      n_jobs = -1))])

param3 = {   
            'lgbm__n_estimators': randint(100,1000), 
            'lgbm__learning_rate': uniform(0.01,0.2),
            'lgbm__max_depth' : randint(3,10),
            'lgbm__boosting' : ['gbdt','rf','dart','goss'],
            'lgbm__colsample_bytree': uniform(0.1,1),
            'lgbm__scale_pos_weight' : uniform(0.1,1)}


clf3 = RandomizedSearchCV(
    pipe3, 
    param_distributions=param3, 
    n_iter=50, 
    cv=3, 
    verbose=1,
    n_jobs=-1
)

clf3.fit(X_train_over, y_train_over)

In [ ]:
model3 = clf3.best_estimator_
y_pred3 = model3.predict(X_test)
RF_opt_fpr3, RF_opt_tpr3, RF_opt_thresholds3 = metrics.roc_curve(y_test, y_pred3)
RF_opt_AUC3 = metrics.auc(RF_opt_fpr3, RF_opt_tpr3)

fig = px.area(
    x=RF_opt_fpr3, y=RF_opt_tpr3,
    title='AUC of Adaboost:'' %0.4f'% RF_opt_AUC3,
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=600
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()